## Lab 1

In [8]:
ALPHABET = ('a', 'ą', 'b', 'c', 'ć', 'd', 'e', 'ę', 'f', 'g',
            'h', 'i', 'j', 'k', 'l', 'ł', 'm', 'n', 'ń', 'o',
            'ó', 'p', 'r', 's', 'ś', 't', 'u', 'w', 'y', 'z',
            'ź', 'ż')

### zad 1.1 Implementacja algorytmu naiwego znajdującego pattern w tekście

In [9]:
def naive_string_maching(text, pattern):
    n = len(text)
    m = len(pattern)
    for s in range(n - m + 1):
        if pattern == text[s : s + m]:
            print(f'Znaleziono na pozycji {s}')

In [10]:
naive_string_maching("aaaaabbbaaaabaaab", "ab")

Znaleziono na pozycji 4
Znaleziono na pozycji 11
Znaleziono na pozycji 15


### Zad 1.2.1 Implemetacja automatu skończonego

In [11]:
def fa_string_maching(text, delta):
    q = 0
    length = len(delta) - 1
    for i in range(0, len(text)):
        q = delta[q] [text[i]]
        if(q == length):
            print(f'Przesunięcie {i + 1 - q} jest poprawne')

### Zad 1.2.2 Implementacja algorytmu generującego funkcję przejścia


In [14]:
def transition_table(pattern):
    result = []
    for q in range(len(pattern) + 1):
        result.append({})
        for a in ALPHABET: #Przechodzenie po alfabecie
            k = min(len(pattern) + 1, q + 2)
            while True:
                k = k - 1
                # q[:k] - prefix
                # q[-k:] - sufix
                if (k == 0 or pattern[:k] == (pattern[:q] + a)[-k:]):
                    break
            result[q][a] = k
    
    return result

### Zad 1.3 Algorytm Knutha-Morrisa-Pratta (KMP)

In [15]:
def prefixFunction(pattern):
    pi = [0]
    k = 0

    for q in range(1, len(pattern)):
    
        while (k > 0 and pattern[k] != pattern[q]):
            k = pi[k-1]
        if(pattern[k] == pattern[q]):
            k = k + 1
        pi.append(k)
    
    return pi

In [17]:
def kmpStringMatching(text, pattern, pi):
    q = 0
    for i in range(0, len(text)):
        while(q > 0 and pattern[q] != text[i]):
            q = pi[q-1]
        if(pattern[q] == text[i]):
            q = q + 1
        if(q == len(pattern)):
            print(f"Przesunięcie {i + 1 - q} jest poprawne")
            q = pi[q-1]

### Zad 2 - testy porównujące szybkość